## Download Data from Google-Drive

In [1]:
'''
File Name : Data
File Link : https://drive.google.com/file/d/1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw/view?usp=share_link
File Id : '1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw'

'''
!gdown --id 1rWS8Jj19ZOzdXFR4jUjmHIx_fTRyYzOw
!unzip ccle_ctrpv2_gdse.zip

'''
File Name : text-mining
File Link : https://drive.google.com/drive/folders/1YTehPDOyXq61biQNtafvmi6M5U4QJbzQ?usp=sharing
File Id : '1YTehPDOyXq61biQNtafvmi6M5U4QJbzQ'

'''

!gdown --id 1YTehPDOyXq61biQNtafvmi6M5U4QJbzQ

'gdown' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'gdown' is not recognized as an internal or external command,
operable program or batch file.


## Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error , mean_absolute_error
from time import perf_counter, sleep
import math
import scipy.stats as stats
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

## Read train and test data

In [6]:
# x_train
ccle_ctrpv2 = pd.read_csv("../data/ccle_ctrpv2_rnaseq_tpm.csv")
# x_test
ccle_ctrpv2_aac = pd.read_csv("../data/ccle_ctrpv2_aac.csv")
# y_train
gdse_rnaseq = pd.read_csv("../data/gdse_rnaseq_tpm.csv")
# y_test
gdse_aac = pd.read_csv("../data/gdse_aac.csv")

## Cleaning the dataframes

In [7]:
drug_names = gdse_aac.columns.values.tolist()
drug_names = drug_names[1:]


ccle_ctrpv2 = ccle_ctrpv2.rename(columns={"Unnamed: 0": "sample"})
gdse_rnaseq = gdse_rnaseq.rename(columns={"Unnamed: 0": "sample"})
ccle_ctrpv2_aac = ccle_ctrpv2_aac.rename(columns={"Unnamed: 0": "sample"})
gdse_aac = gdse_aac.rename(columns={"Unnamed: 0": "sample"})

names = []
for i in range(0, 40):
    names.append(f'Column{i}')

In [8]:
def remove_nan(data):
    data = data.dropna()
    data.reset_index(drop = True)
    return data

In [9]:
'''
Initializing k-fold cross-validation.

n_splits=4: the dataset will be split into 4 equal-sized folds or subsets.

shuffle: data will be shuffled before splitting into folds, 
         ensuring randomization in the distribution of samples across each fold.

'''
kfold = KFold(n_splits=4, shuffle=True)


'''
Initializing StandardScaler to normalize the data to have mean of 0 and variance of 1.
'''
Sscaler = StandardScaler()

## Creating a shallow neural network
---
1. **Layers**

First Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Second Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Third Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>
Fourth Hidden Layer -> **Activation Function:** *Rectified Linear Unit (Relu)* <br>

Output Layer -> **Activation Function:** *Sigmoid* <br>

Sigmoid activation function is used to force the output to have a range between 0 and 1, 
as the desired output (Drug Response) has a value between 0 and 1.

2. **Optimizer**

Using *Stochastic Gradient descent* (SGD) with momentum:

        learning rate: 0.01
        momentum = 0.9

3. **Loss Function**

Using *mean squared error* (MSE) for the loss function

4. **Regularization technique**

20% dropout after each hidden layer 

5. **Network schematic**

![MarineGEO circle logo](Arch.png)

In [10]:
model = tf.keras.models.Sequential()

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1 , activation='sigmoid'))


learning_rate = 0.01
momentum = 0.9
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate, momentum)

optimizer_rmsprop = tf.keras.optimizers.RMSprop()

In [11]:
model.build((None, 19924))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               5100800   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 64)                4

# Training procedure

*For each drug in the drug list:*
   1. Split the data in to train and test sets, and then remove the Nan values.
   2. Merging X_train with ytrain, and X_test with  y_test because the samples in each are different.
   3. Normalize the training X. </br>
   
   *Running the following 6 times:*
   
       4. Applying 4-fold Cross Valisation
       5. Applying the relevant features selection technique, and extract the selected features.
       6. Train the model based on the CV data and extracted features
       7. Saving the best model performed in CV and the best features
       
   8. Predicting the test set using the best model from the previous section
   9. Calculating the correlation score between the predicted drug_respons and the actual drug_response

In [13]:
cor_df = pd.DataFrame([], columns = ['corr'])

correlatoins = []
correlatoins = []
mse_train_list = []
rmse_train_list = []
mae_train_list = []
spearmanr_correlation_train_list = []
kendalltau_correlation_train_list = []
cr = []
mse = []
rmse = []
mae = []
spearmanr =[]
kendalltau = []
start = perf_counter()
for drug in drug_names:
    text_mining_drug = pd.read_csv("../data/text_mining_feature/"+drug+".csv",  names = names)
    text_mining_drug = text_mining_drug.tail(-1)

         
    selected_features = []
    best_model_top_features = set()
    model_performances = []
    model_acc = []
    pearson_correlations = []
    top_features = []
    merge_features = []
    new_features = text_mining_drug['Column3']
    
    
    model = tf.keras.models.Sequential()
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1 , activation='sigmoid'))


    learning_rate = 0.01
    momentum = 0.9
    optimizer_sgd = tf.keras.optimizers.SGD(learning_rate, momentum)

    optimizer_rmsprop = tf.keras.optimizers.RMSprop()

    '''
        train test split
    '''
    
    y_train = ccle_ctrpv2_aac[['sample', drug]]
    y_test  = gdse_aac[['sample', drug]]

    y_train = remove_nan(y_train)
    y_test  = remove_nan(y_test)

    ccle_ctrpv2 = remove_nan(ccle_ctrpv2)
    gdse_rnaseq = remove_nan(gdse_rnaseq)


    '''
    Merge
    '''

    gdse_rnaseq_merged_df = pd.merge(gdse_rnaseq, y_test, on='sample', how='inner')
    ccle_ctrpv2_merged_df = pd.merge(ccle_ctrpv2, y_train, on='sample', how='inner')

    X_train = ccle_ctrpv2_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]
    X_test  = gdse_rnaseq_merged_df.iloc[:, 1: len(gdse_rnaseq.columns)]

    Y_train = ccle_ctrpv2_merged_df.iloc[:, len(gdse_rnaseq.columns):]
    Y_test  = gdse_rnaseq_merged_df.iloc[:, len(gdse_rnaseq.columns):]

    X_train_norm = Sscaler.fit_transform(X_train)
    X_train_normalized = pd.DataFrame(X_train_norm, columns=X_train.columns)

    
    merge_features = [item for item in np.asarray(new_features) if item in X_train_normalized.columns]

    for i in range(6):
            
        selected_features = []
        best_model_top_features = set()
        model_performances = []
        model_acc = []
        pearson_correlations = []
        best_model_path = 'saved_models/best_model_'+ drug + str(i)
        
        
        for train, validation in kfold.split(X_train_normalized , Y_train):

            train_data = X_train_normalized.iloc[train]
            train_labels = Y_train.iloc[train]
            val_data = X_train_normalized.iloc[validation]
            val_labels = Y_train.iloc[validation]




            top_features = merge_features


            train_data_selected = train_data[np.asarray(top_features)]
            val_data_selected = val_data[np.asarray(top_features)]


            model.compile(optimizer = optimizer_sgd, loss='mean_squared_error', metrics='MAE')

            checkpoint_callback = ModelCheckpoint(filepath=best_model_path+ '.h5' , monitor='MAE', save_best_only=True, mode='min')

            history = model.fit(train_data_selected, train_labels, epochs=15,callbacks=[checkpoint_callback], validation_data=(val_data_selected, val_labels))


        new_X = X_test[top_features]
        Xtest_norm = Sscaler.fit_transform(new_X)
        Xtest_normalized = pd.DataFrame(Xtest_norm, columns=new_X.columns)

        loaded_model = tf.keras.models.load_model(best_model_path+ '.h5' )

        gdsc_predictions = loaded_model.predict(Xtest_normalized)

        Y_test_array = np.asarray(Y_test)

        correlation, _ = pearsonr(Y_test_array.flatten(),  gdsc_predictions.flatten())
        mse_train = mean_squared_error(Y_test_array.flatten(), gdsc_predictions.flatten())
        rmse_train = math.sqrt(mse_train)
        mae_train = mean_absolute_error(Y_test_array.flatten(), gdsc_predictions.flatten())

    # Calculate Correlation for the validation data
        spearmanr_correlation_train, _ = stats.spearmanr(Y_test_array.flatten(), gdsc_predictions.flatten())
        kendalltau_correlation_train, _ = stats.kendalltau(Y_test_array.flatten(), gdsc_predictions.flatten())
    
        correlatoins.append(correlation)
        mse_train_list.append(mse_train)
        rmse_train_list.append(rmse_train)
        mae_train_list.append(mae_train)
        spearmanr_correlation_train_list.append(spearmanr_correlation_train)
        kendalltau_correlation_train_list.append(kendalltau_correlation_train)
        
        
    index = correlatoins.index(max(correlatoins))
    cr.append(max(correlatoins))
    mse.append(mse_train_list[index])
    rmse.append(rmse_train_list[index])
    mae.append(mae_train_list[index])
    spearmanr.append(spearmanr_correlation_train_list[index])
    kendalltau.append(kendalltau_correlation_train_list[index])
    correlatoins = []
    mse_train_list = []
    rmse_train_list = []
    mae_train_list = []
    spearmanr_correlation_train_list = []
    kendalltau_correlation_train_list = []
    
    
    
end = perf_counter()

Epoch 1/15
18/18 [==============================] - 1s 20ms/step - loss: 0.0455 - MAE: 0.1651 - val_loss: 0.0153 - val_MAE: 0.1004
Epoch 2/15
18/18 [==============================] - 0s 12ms/step - loss: 0.0190 - MAE: 0.1149 - val_loss: 0.0076 - val_MAE: 0.0690
Epoch 3/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0139 - MAE: 0.0937 - val_loss: 0.0079 - val_MAE: 0.0687
Epoch 4/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0117 - MAE: 0.0870 - val_loss: 0.0070 - val_MAE: 0.0640
Epoch 5/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0114 - MAE: 0.0847 - val_loss: 0.0070 - val_MAE: 0.0643
Epoch 6/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0096 - MAE: 0.0766 - val_loss: 0.0076 - val_MAE: 0.0673
Epoch 7/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0103 - MAE: 0.0804 - val_loss: 0.0073 - val_MAE: 0.0657
Epoch 8/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0099 - MAE: 0.

18/18 [==============================] - 0s 6ms/step - loss: 0.0035 - MAE: 0.0464 - val_loss: 0.0030 - val_MAE: 0.0461
Epoch 4/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0032 - MAE: 0.0445 - val_loss: 0.0031 - val_MAE: 0.0470
Epoch 5/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0036 - MAE: 0.0461 - val_loss: 0.0031 - val_MAE: 0.0472
Epoch 6/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0031 - MAE: 0.0437 - val_loss: 0.0030 - val_MAE: 0.0462
Epoch 7/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0033 - MAE: 0.0447 - val_loss: 0.0029 - val_MAE: 0.0448
Epoch 8/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0030 - MAE: 0.0436 - val_loss: 0.0030 - val_MAE: 0.0456
Epoch 9/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0033 - MAE: 0.0445 - val_loss: 0.0031 - val_MAE: 0.0467
Epoch 10/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0034 - MAE: 0.0449 - val_l

18/18 [==============================] - 0s 6ms/step - loss: 0.0024 - MAE: 0.0376 - val_loss: 0.0026 - val_MAE: 0.0465
Epoch 6/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0024 - MAE: 0.0383 - val_loss: 0.0024 - val_MAE: 0.0442
Epoch 7/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0023 - MAE: 0.0367 - val_loss: 0.0022 - val_MAE: 0.0430
Epoch 8/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0024 - MAE: 0.0376 - val_loss: 0.0023 - val_MAE: 0.0435
Epoch 9/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0024 - MAE: 0.0379 - val_loss: 0.0024 - val_MAE: 0.0445
Epoch 10/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0022 - MAE: 0.0376 - val_loss: 0.0025 - val_MAE: 0.0459
Epoch 11/15
18/18 [==============================] - 0s 7ms/step - loss: 0.0025 - MAE: 0.0382 - val_loss: 0.0026 - val_MAE: 0.0463
Epoch 12/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0023 - MAE: 0.0377 - val

18/18 [==============================] - 0s 6ms/step - loss: 0.0016 - MAE: 0.0318 - val_loss: 0.0021 - val_MAE: 0.0430
Epoch 8/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0326 - val_loss: 0.0020 - val_MAE: 0.0415
Epoch 9/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0330 - val_loss: 0.0020 - val_MAE: 0.0419
Epoch 10/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0323 - val_loss: 0.0020 - val_MAE: 0.0417
Epoch 11/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0019 - MAE: 0.0335 - val_loss: 0.0020 - val_MAE: 0.0421
Epoch 12/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0332 - val_loss: 0.0021 - val_MAE: 0.0423
Epoch 13/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0018 - MAE: 0.0331 - val_loss: 0.0021 - val_MAE: 0.0430
Epoch 14/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0017 - MAE: 0.0325 - v

18/18 [==============================] - 0s 6ms/step - loss: 0.0015 - MAE: 0.0296 - val_loss: 0.0022 - val_MAE: 0.0440
Epoch 10/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0013 - MAE: 0.0288 - val_loss: 0.0021 - val_MAE: 0.0431
Epoch 11/15
18/18 [==============================] - 0s 8ms/step - loss: 0.0014 - MAE: 0.0287 - val_loss: 0.0023 - val_MAE: 0.0445
Epoch 12/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0017 - MAE: 0.0313 - val_loss: 0.0022 - val_MAE: 0.0434
Epoch 13/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0015 - MAE: 0.0309 - val_loss: 0.0023 - val_MAE: 0.0444
Epoch 14/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0015 - MAE: 0.0300 - val_loss: 0.0023 - val_MAE: 0.0451
Epoch 15/15
18/18 [==============================] - 0s 6ms/step - loss: 0.0015 - MAE: 0.0311 - val_loss: 0.0023 - val_MAE: 0.0444
Epoch 1/15
18/18 [==============================] - 1s 17ms/step - loss: 0.0017 - MAE: 0.0327 -

...

17/17 [==============================] - 0s 7ms/step - loss: 0.0043 - MAE: 0.0521 - val_loss: 0.0029 - val_MAE: 0.0440
Epoch 6/15
17/17 [==============================] - 0s 11ms/step - loss: 0.0041 - MAE: 0.0509 - val_loss: 0.0030 - val_MAE: 0.0455
Epoch 7/15
17/17 [==============================] - 0s 12ms/step - loss: 0.0043 - MAE: 0.0509 - val_loss: 0.0032 - val_MAE: 0.0477
Epoch 8/15
17/17 [==============================] - 0s 8ms/step - loss: 0.0043 - MAE: 0.0512 - val_loss: 0.0032 - val_MAE: 0.0473
Epoch 9/15
17/17 [==============================] - 0s 14ms/step - loss: 0.0038 - MAE: 0.0479 - val_loss: 0.0032 - val_MAE: 0.0478
Epoch 10/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0044 - MAE: 0.0521 - val_loss: 0.0030 - val_MAE: 0.0455
Epoch 11/15
17/17 [==============================] - 0s 11ms/step - loss: 0.0039 - MAE: 0.0485 - val_loss: 0.0032 - val_MAE: 0.0473
Epoch 12/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0039 - MAE: 0.0492 -

17/17 [==============================] - 0s 10ms/step - loss: 0.0029 - MAE: 0.0421 - val_loss: 0.0021 - val_MAE: 0.0426
Epoch 8/15
17/17 [==============================] - 0s 11ms/step - loss: 0.0029 - MAE: 0.0418 - val_loss: 0.0023 - val_MAE: 0.0440
Epoch 9/15
17/17 [==============================] - 0s 11ms/step - loss: 0.0034 - MAE: 0.0457 - val_loss: 0.0024 - val_MAE: 0.0450
Epoch 10/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0032 - MAE: 0.0431 - val_loss: 0.0022 - val_MAE: 0.0425
Epoch 11/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0029 - MAE: 0.0430 - val_loss: 0.0025 - val_MAE: 0.0457
Epoch 12/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0033 - MAE: 0.0437 - val_loss: 0.0025 - val_MAE: 0.0462
Epoch 13/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0034 - MAE: 0.0450 - val_loss: 0.0024 - val_MAE: 0.0453
Epoch 14/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0033 - MAE: 0.0440

17/17 [==============================] - 0s 10ms/step - loss: 0.0024 - MAE: 0.0390 - val_loss: 0.0020 - val_MAE: 0.0395
Epoch 10/15
17/17 [==============================] - 0s 12ms/step - loss: 0.0026 - MAE: 0.0389 - val_loss: 0.0019 - val_MAE: 0.0388
Epoch 11/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0024 - MAE: 0.0386 - val_loss: 0.0020 - val_MAE: 0.0392
Epoch 12/15
17/17 [==============================] - 0s 15ms/step - loss: 0.0025 - MAE: 0.0386 - val_loss: 0.0018 - val_MAE: 0.0377
Epoch 13/15
17/17 [==============================] - 0s 10ms/step - loss: 0.0024 - MAE: 0.0390 - val_loss: 0.0018 - val_MAE: 0.0373
Epoch 14/15
17/17 [==============================] - 0s 14ms/step - loss: 0.0025 - MAE: 0.0394 - val_loss: 0.0019 - val_MAE: 0.0386
Epoch 15/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0023 - MAE: 0.0376 - val_loss: 0.0020 - val_MAE: 0.0395
Epoch 1/15
17/17 [==============================] - 1s 24ms/step - loss: 0.0025 - MAE: 0.0

17/17 [==============================] - 0s 10ms/step - loss: 0.0019 - MAE: 0.0345 - val_loss: 0.0019 - val_MAE: 0.0412
Epoch 12/15
17/17 [==============================] - 0s 10ms/step - loss: 0.0023 - MAE: 0.0380 - val_loss: 0.0020 - val_MAE: 0.0421
Epoch 13/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0022 - MAE: 0.0357 - val_loss: 0.0020 - val_MAE: 0.0422
Epoch 14/15
17/17 [==============================] - 0s 16ms/step - loss: 0.0018 - MAE: 0.0328 - val_loss: 0.0021 - val_MAE: 0.0430
Epoch 15/15
17/17 [==============================] - 0s 10ms/step - loss: 0.0023 - MAE: 0.0368 - val_loss: 0.0020 - val_MAE: 0.0417
Epoch 1/15
17/17 [==============================] - 1s 36ms/step - loss: 0.0018 - MAE: 0.0332 - val_loss: 0.0017 - val_MAE: 0.0375
Epoch 2/15
17/17 [==============================] - 0s 12ms/step - loss: 0.0019 - MAE: 0.0347 - val_loss: 0.0017 - val_MAE: 0.0377
Epoch 3/15
17/17 [==============================] - 0s 7ms/step - loss: 0.0020 - MAE: 0.035

17/17 [==============================] - 0s 19ms/step - loss: 0.0018 - MAE: 0.0323 - val_loss: 0.0017 - val_MAE: 0.0375
Epoch 14/15
17/17 [==============================] - 0s 19ms/step - loss: 0.0017 - MAE: 0.0328 - val_loss: 0.0016 - val_MAE: 0.0362
Epoch 15/15
17/17 [==============================] - 0s 11ms/step - loss: 0.0018 - MAE: 0.0331 - val_loss: 0.0016 - val_MAE: 0.0359
Epoch 1/15
17/17 [==============================] - 1s 19ms/step - loss: 0.0020 - MAE: 0.0347 - val_loss: 0.0015 - val_MAE: 0.0358
Epoch 2/15
17/17 [==============================] - 0s 13ms/step - loss: 0.0017 - MAE: 0.0321 - val_loss: 0.0017 - val_MAE: 0.0381
Epoch 3/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0018 - MAE: 0.0331 - val_loss: 0.0017 - val_MAE: 0.0376
Epoch 4/15
17/17 [==============================] - 0s 14ms/step - loss: 0.0019 - MAE: 0.0338 - val_loss: 0.0017 - val_MAE: 0.0381
Epoch 5/15
17/17 [==============================] - 0s 9ms/step - loss: 0.0019 - MAE: 0.0334 

# Saving the Results

In [14]:
execution_time = end - start

df = pd.DataFrame({"Drugs": drug_names, "pearsonCor": cr , "spearmanCor": spearmanr , "kendallCor": kendalltau , 
                  "RMSE": rmse ,  "MSE": mse , "MAE": mae , "Time": execution_time})
df.to_csv('DL-results/Result_textmining.csv', index=False)